In [84]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'pillow'

In [88]:
from PIL import Image

In [5]:
# Keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.datasets import mnist
# https://keras.io/datasets/

In [6]:
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [9]:
test_df = pd.read_csv('test_dataset.csv')
train_df = pd.read_csv('train_dataset.csv')

- SHUFFLE DATA FRAME

In [10]:
train_df=train_df.sample(frac=1)

- Get Targets in a series

In [11]:
Class = train_df.Target

- Get Image Paths from CSV column(patientID)

In [53]:
image_path = []
for i in test_df.patientId:
    image_path.append('images/%s.dcm' % i)

len(image_path)

16928

- Read images with pydicom

In [54]:
images = []
for i in image_path:
    images.append(pydicom.read_file(i))

In [55]:

images[0].pixel_array

array([[  1,   1,   1, ...,   1,   1,   1],
       [  1,   1,   1, ...,   1,   1,   1],
       [  1,   1,   1, ...,   1,   1,   1],
       ...,
       [ 91, 167, 167, ...,  45,  46,  25],
       [106, 170, 167, ...,  46,  50,  32],
       [ 42,  90,  79, ...,  19,  25,  10]], dtype=uint8)

In [56]:
pixel_values =[]
def export_pixel_arrays_to_csv():
    for i in range(16928):
        row = []
        row.append(test_df.patientId[i])
        row.append(images[i].pixel_array)
        row.append(test_df.Target[i])
        pixel_values.append(row)

In [59]:
export_pixel_arrays_to_csv()

In [133]:
pixel_df = pd.DataFrame(pixel_values)
# 'patient_id':pixel_values[0], 'pixel_array':pixel_values[1], 'opacity':pixel_values[2])


In [141]:
new_pixel_df = pixel_df.rename(index=int, columns={0: "patient_id", 1: "pixel_array", 2: "opacity"})

array([0, 1, 2])

In [143]:
new_pixel_df.head(5)

,patient_id,pixel_array,opacity
0,8aa2f1cd-a9f5-4f10-8dd7-1d40b2fbaf50,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",1
1,fcb06a43-8652-4bbf-bdf8-bb961137302a,"[[53, 110, 90, 95, 95, 95, 95, 95, 95, 94, 93,...",1
2,6d1efe52-1513-43c8-947f-158a33c0ad81,"[[21, 38, 17, 11, 8, 5, 5, 2, 2, 2, 2, 1, 1, 1...",1
3,d7545025-6654-4cc3-83d1-99a34476c72a,"[[2, 4, 5, 5, 4, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4,...",1
4,bfbf1dcd-242c-4f22-8ef3-f72c8295535f,"[[229, 226, 224, 223, 221, 218, 216, 216, 213,...",1


In [153]:
# new_pixel_df.groupby("opacity").count()
zeros = new_pixel_df.loc[new_pixel_df["opacity"] == 0]
ones = new_pixel_df.loc[new_pixel_df["opacity"] == 1]
# zero_sample = zero.sample(8964)
# zero_sample_test = zero_sample.sample(500)
# new_df = zero_sample.append(zero_sample_test)
# testing_zeros = new_df.drop_duplicates(keep = False)

print(len(zeros))
print(len(ones))

8464
8464


In [154]:
zeros.head(5)

,patient_id,pixel_array,opacity
8464,71ea8f7f-d36d-47a5-89c3-7ec145a201c2,"[[178, 165, 151, 138, 124, 112, 102, 93, 85, 7...",0
8465,b40f6a18-fbfd-4c4f-851a-8bd09dc8fe3b,"[[0, 1, 4, 6, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,...",0
8466,6876141d-e5f2-4523-a0ae-c6d0a8f30169,"[[33, 32, 14, 21, 61, 58, 54, 20, 0, 6, 2, 0, ...",0
8467,84030ab5-8cd1-4db4-9b70-df83042cc856,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0
8468,b6cf53c3-173c-421e-9991-eeea32ef7c61,"[[223, 216, 214, 216, 216, 215, 213, 209, 210,...",0


In [183]:
patient_list = zeros['patient_id'].tolist()
pixel_array_list = zeros['pixel_array'].tolist()
opacity_list = zeros['opacity'].tolist()


In [184]:
len(pixel_array_list)

8464

In [185]:
def export_no_opac_jpg():
    for i in range(len(pixel_array_list)):
        Image.fromarray(pixel_array_list[i]).save(f"jpg_xrays/no_opacity/{patient_list[i]}_{i}.jpg")

In [169]:
Image.fromarray(pixel_array_list[0]).save("jpg_xrays/no_opacity/71ea8f7f-d36d-47a5-89c3-7ec145a201c2.jpg")

In [186]:
export_no_opac_jpg()

In [187]:
patient_list = ones['patient_id'].tolist()
pixel_array_list = ones['pixel_array'].tolist()
opacity_list = ones['opacity'].tolist()

In [188]:
len(pixel_array_list)

8464

In [189]:
def export_opac_jpg():
    for i in range(len(pixel_array_list)):
        Image.fromarray(pixel_array_list[i]).save(f"jpg_xrays/opacity/{patient_list[i]}_{i}.jpg")

In [190]:
export_opac_jpg()

- Get Pixel array with (pydicom.pixel_array)

In [17]:
pixel_s = []
for i in range(len(images)):
    pixel_s.append(images[i].pixel_array)

- Scale Images
- scale = 1.00% = 1054x1054

- How many unique IDs/x-rays did we load

In [18]:
len(pixel_s)

1000

- Scale Images
- Scale = %1.00= 1054 X 1054

In [11]:
scaled = []
image_scale = 1
for i in range(len(pixel_s)):
    scaled.append(rescale(pixel_s[i],scale=image_scale,mode='reflect',multichannel=False))

C:\Users\James\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [12]:
width = len(scaled[0])
height = len(scaled[1])
print("IMAGE SIZE: ",height, width)

IMAGE SIZE:  1024 1024


- Make giant NUMPY ARRAY of all images and all pixels

In [13]:
X_train=np.array(scaled)
X_train.shape

(1000, 1024, 1024)

- Class variable holds our target values
- .values takes pandas.series => numpy array

In [14]:
y_train = Class.values

- reshape to be [samples][pixels][width][height]


In [15]:
X_train = X_train.reshape(X_train.shape[0], width, height,1 ).astype('float32') # for the Convnet

In [16]:
X_train.shape

(1000, 1024, 1024, 1)

- normalize inputs from 0-255 to 0-1

In [17]:
X_train = X_train / 255

- one hot encode outputs

In [18]:
y_train = to_categorical(y_train)
y_train.shape

(1000, 2)

In [19]:
int(time.time())

1540300850

In [20]:
num_classes = y_train.shape[1]
num_classes

2

In [21]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(width, height, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# save logs for tensorboard
tensorboard = TensorBoard(log_dir=f'logs/{int(time.time())}_{width}x{height}')

In [22]:
filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

In [ ]:
# Fit (train) the model
model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2,
    validation_split = .2,
    callbacks=[tensorboard, checkpoint]
)

Train on 800 samples, validate on 200 samples
Epoch 1/10


In [ ]:
#tensorboard --logdir=logs
#http://DESKTOP-D2C2TFC:6006